In [ ]:
import config
import lib_db
import lib_nlp
import datetime
import lib_vk
import datetime
import lib_process
from random import shuffle
import pandas as pd

In [ ]:
from_datetime = datetime.datetime(2017,7,25,0,0,0)

In [ ]:
data = {}
import gzip, json, pickle
datafile = 'saved_data.pkl'
def save_data():
    global data
    with open(datafile, 'wb') as f:
        pickle.dump(data, f)
        
def load_data():
    global data
    with open(datafile, 'rb') as f:
        data = pickle.load(f)

In [ ]:
load_data()
data[name] = {}

In [ ]:
import vk_api
access_token = ''
vk_session = vk_api.VkApi(token=access_token)
vk = vk_session.get_api()
lib_vk.vk = vk

In [ ]:
def get_posts_vk(domain):
    vkposts = []
    cur_offset = 0
    chunk_size = 100
    while True:
        chunk = vk.wall.get(domain=domain, count=chunk_size, offset=cur_offset, v=5.68)
        chunk = chunk['items']
        if len(chunk) == 0:
            break
        for post in chunk:
            post_date = datetime.datetime.fromtimestamp(int(post['date']))
            if post_date < from_datetime and post.get('is_pinned', 0) != 1:
                return vkposts
            if post_date >= from_datetime:
                vkposts.append(post)
        cur_offset += chunk_size
    return vkposts

In [ ]:
vkposts = get_posts_vk(urls[name]['vk'])

In [ ]:
group_info = lib_vk.get_group_info(urls[name]['vk'])

In [ ]:
%%time
comment_objects = []
for p in vkposts:
    owner_id = p['owner_id']
    post_id = p['id']
    chunk = vk.wall.getComments(owner_id=owner_id, post_id=post_id, count=100, sort='desc', v=5.68)
    for c in chunk['items']:
        if len(c['text']) > 4:
            c['sentiment'] = lib_process.get_sentiment(c['text'], reports_mode=True)
        else:
            c['sentiment'] = 'unknown'
        comment_objects.append(c)

In [ ]:
likes = 0
reposts = 0
comments = 0
views = 0
for p in vkposts:
    likes += p['likes'].get('count', 0)
    reposts += p['reposts'].get('count', 0)
    comments += p['comments'].get('count', 0)
    views += p.get('views', {}).get('count', 0)

In [ ]:
len(vkposts), likes, comments, reposts, views, group_info['members_count']

In [ ]:
data[name]['vk'] = {'post_count': len(vkposts), 'likes': likes, 'comments': comments, 'reposts': reposts, 'views': views, 'comments_objects': comment_objects, 'posts': vkposts, 'subscribers_count': group_info['members_count']}
# 'reliability_data': rel_data, 'reliability_rating': rel_rating

In [ ]:
import facebook
import requests
import dateutil.parser as dateparser
import pytz
utc=pytz.UTC
graph = facebook.GraphAPI(config.TOKEN_FB_PROD)

In [ ]:
def get_posts_fb(fb_url):
    profile = graph.get_object(fb_url, fields='fan_count,location,picture,name')
    fbposts = []
    chunk = graph.get_connections(profile['id'], 'posts', fields='from,created_time,message,id,shares,likes.summary(true),comments.summary(true)')
    while True:
        try:
            for post in chunk['data']:
                fbposts.append(post)
                parsed_time = dateparser.parse(post['created_time']).replace(tzinfo=utc)
                if parsed_time < from_datetime.replace(tzinfo=utc):
                    return fbposts, profile
            chunk = requests.get(chunk['paging']['next']).json()
        except KeyError:
            break
    return fbposts, profile

In [ ]:
fbposts, profile = get_posts_fb(urls[name]['fb'])

In [ ]:
profile

In [ ]:
likes = 0
reposts = 0
comments = 0
for p in fbposts:
    likes += p.get('likes').get('summary').get('total_count')
    reposts += p.get('shares', {}).get('count', 0)
    comments += p.get('comments').get('summary').get('total_count')

In [ ]:
len(fbposts), likes, comments, reposts

In [ ]:
data[name]['fb'] = {'post_count': len(fbposts), 'likes': likes, 'comments': comments, 'reposts': reposts, 'posts': fbposts, 'subscribers_count': profile['fan_count']}

In [ ]:
import tweepy
import urllib
import base64
import json
import urllib.parse
import urllib.request

class AppAuthHandler(tweepy.auth.AuthHandler):
    TOKEN_URL = 'https://api.twitter.com/oauth2/token'

    def __init__(self, consumer_key, consumer_secret):
        token_credential = urllib.parse.quote(consumer_key) + ':' + urllib.parse.quote(consumer_secret)
        credential = base64.b64encode(token_credential.encode())

        value = {'grant_type': 'client_credentials'}
        data = urllib.parse.urlencode(value)
        req = urllib.request.Request(self.TOKEN_URL)
        req.add_header('Authorization', 'Basic '.encode() + credential)
        req.add_header('Content-Type', 'application/x-www-form-urlencoded;charset=UTF-8')

        response = urllib.request.urlopen(req, data.encode())
        json_response = json.loads(response.read().decode())
        self._access_token = json_response['access_token']

    def apply_auth(self, url, method, headers, parameters):
        headers['Authorization'] = 'Bearer ' + self._access_token

In [ ]:
CONSUMER_KEY = ''
CONSUMER_SECRET = ''

auth = tweepy.AppAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
api = tweepy.API(auth)

In [ ]:
already_processed = {}

In [ ]:
def find_original_tweet(tweet, client_name):
    try:
        client_first_reply_date = None
        while tweet.in_reply_to_status_id:
            p_date = tweet.created_at
            if tweet.author.screen_name == client_name:
                client_first_reply_date = p_date
            tweet = api.get_status(tweet.in_reply_to_status_id)
        if already_processed.get(tweet.id):
            return
        already_processed[tweet.id] = True
        print('----')
        p_date = tweet.created_at
        print('время ответа: ', client_first_reply_date-p_date)
        print('twitter.com/{}/status/{}'.format(tweet.author.screen_name, tweet.id))
        print(tweet.text)
    except:
        pass

In [ ]:
def get_posts_twitter(user_id):
    twposts = []
    max_id = False
    chunk_size = 100
    while True:
        if max_id:
            chunk = api.user_timeline(screen_name=user_id, count=100, include_rts=False, max_id=max_id)
        else:
            chunk = api.user_timeline(screen_name=user_id, count=100, include_rts=False)
        if len(chunk) == 0:
            break
        for post in chunk:
            post_date = post.created_at
            if post_date < from_datetime:
                return twposts
            if not post.in_reply_to_status_id:
                twposts.append(post)
            else:
                find_original_tweet(post, user_id)
            max_id = post.id
    return twposts

In [ ]:
tweets = get_posts_twitter(urls[name]['twitter'])

In [ ]:
likes = 0
reposts = 0
for p in tweets:
    likes += p.favorite_count
    reposts += p.retweet_count

In [ ]:
len(tweets), likes, reposts

In [ ]:
tweets[0].author.followers_count

In [ ]:
data[name]['twitter'] = {'post_count': len(tweets), 'likes': likes, 'reposts': reposts, 'posts': tweets, 'subscribers_count': tweets[0].author.followers_count}

In [ ]:
save_data()